In [1]:
import speech_recognition as sr 
import pandas as pd
import PyPDF2
import string
from pathlib import *
from gtts import gTTS
import os
import glob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
import pyttsx3 

In [2]:
def recorder(): 
    r = sr.Recognizer()                 # initialize recognizer
    with sr.Microphone() as source:     # mention source it will be either Microphone or audio files.
        print("Speak your query :")
        audio = r.listen(source)        # listen to the source
        try:
            speech = r.recognize_google(audio)    # use recognizer to convert our audio into text part.
            
            print("You said : {}".format(speech))
        except:
            print("Sorry could not recognize your voice")
    return(speech)

In [3]:
def cleanspeech(speech):
    
    speech=speech.lower() #lowercase
    
    punct = string.punctuation #remve punctuations
    trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
    speech=speech.translate(trantab)
    
    stopwords_list = stopwords.words('english') #remove stopwords
    whitelist=['during','before','after','above','below','up','down','no','not']
    words = speech.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
    speech=" ".join(clean_words)
    lemmatizer = WordNetLemmatizer()
    word_list=word_tokenize(speech)
    speech= [lemmatizer.lemmatize(w) for w in word_list]
    
    return (speech)

In [4]:
def cleandocument(data1):
    
    data1[0]=data1[0].str.lower()
    punct = string.punctuation #remve punctuations
    trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
    data1[1]=data1[0].str.translate(trantab) 
    def replace(word):
        return(word.replace('\n',' '))
    data1[1]=data1.apply(lambda x:replace(x[1]),axis=1)
    stop = stopwords.words('english')
    data1[1] = data1[1].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    w_tokenizer = WhitespaceTokenizer()
    lemmatizer = WordNetLemmatizer()
    def lemmatize_text(text):
        return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

    data1[1] = data1[1].apply(lemmatize_text)
    
    return (data1)

In [5]:
def loadpdf():
    data=pd.DataFrame()
    dir_path = Path('F:/NOTES/sem7- bda ibm/cases')
    pdf_files = dir_path.glob('*.pdf')
    i=0
    for files in pdf_files:
        pdfFileObj = open(files,'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        num_pages = pdfReader.numPages
        count = 0
        data.loc[i,0] = ""
        while count < num_pages:
            pageObj = pdfReader.getPage(count)
            count +=1
            data.loc[i,0] += pageObj.extractText()
            #This if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.
            if data.loc[i,0] != "":
               data.loc[i,0] = data.loc[i,0]
            #If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text
            else:
               data.loc[i,0] = textract.process(fileurl, method='tesseract', language='eng')        
        i=i+1
        
    return(data)

In [6]:
def text_to_speech(output):
    for k in range(len(output)):
        engine = pyttsx3.init() 
        engine.say(output.loc[k,0]) 
        engine.runAndWait() 
    return()

In [7]:
def searching_query(speech,data2):
    data3=pd.DataFrame()
    querylen=len(speech)
    i=j=t=0
    for j in range(len(data2)):
        for i in range(querylen):
            if (speech[i] in  data2['keywords'].loc[j]):
                data3.loc[t,0]=data2['data'].loc[j]
        t=t+1
    return (data3)

In [9]:
query=recorder()
speech=cleanspeech(query)
print(speech)

Speak your query :
You said : Supreme India
['supreme', 'india']


In [10]:
data1=pd.DataFrame()
data1=loadpdf()
data2=cleandocument(data1)
data2=data2.rename(columns={0: "data", 1: "keywords"})  
data2

,data,keywords
0,non-reportable\nin the supreme court of india\...,"[non, reportable, supreme, court, india, crimi..."
1,1\n \n \nreportable\n \n \n \n ...,"[1, reportable, supreme, court, india, civil, ..."


In [11]:
output=searching_query(speech,data2)
output.reset_index(inplace=True)
output

,index,0
0,0,non-reportable\nin the supreme court of india\...
1,1,1\n \n \nreportable\n \n \n \n ...


In [ ]:
text_to_speech(output)